In [ ]:

import pandas as pd
import numpy as np

import time
import pickle

from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.datasets import make_blobs, make_moons, load_iris, load_digits
from sklearn.cluster import *
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import homogeneity_score, homogeneity_completeness_v_measure
from sklearn.preprocessing import scale
from collections import Counter

import hdbscan
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('seaborn')

In [ ]:
#load in the data from the pickle objects
all_schools = pd.read_pickle("allschools_df.pkl")
public_schools = pd.read_pickle("public_df.pkl")
for_profit = pd.read_pickle("forprofit_df.pkl")
non_profit = pd.read_pickle("nonprofit_df.pkl")

allschoolsPCA = pd.read_pickle("allschoolsPCA.pkl")
publicPCA = pd.read_pickle("publicschoolPCA.pkl")
nonprofitPCA = pd.read_pickle("nonprofitPCA.pkl")
forprofitPCA = pd.read_pickle("forprofitPCA.pkl")

### HDBSCAN with Principal Components of All Schools

In [ ]:
%%time
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=25)
cluster_labels = clusterer.fit_predict(allschoolsPCA)
hdlabels = clusterer.labels_

In [ ]:
clusterList = list(zip(hdlabels, allschoolsPCA.index.values))
len(clusterList)

In [ ]:

for i in sorted(clusterList, key=lambda x: x[0]):
    if i[0] < 0:
        clusterList.remove(i)

In [ ]:
len(clusterList)

In [ ]:
sorted_ClusterList = sorted(clusterList, key=lambda x: x[0])
#sorted_ClusterList

In [ ]:
# this cell creates a rotating 3D scatter plot of the top 3 principal components in a pop up window
from matplotlib import interactive
from PyQt5 import QtGui, QtCore
interactive(True)
%matplotlib qt

num_clusters = clusterer.labels_.max()
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

color_palette = sns.color_palette('bright', 8)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.9, 0.9, 0.9)
                  for x in clusterer.labels_]
cluster_member_colors = [sns.desaturate(x, p) for x, p in
                         zip(cluster_colors, clusterer.probabilities_)]
ax.scatter(allschoolsPCA.PC1, allschoolsPCA.PC2, allschoolsPCA.PC3, linewidth=0, 
                  c=cluster_member_colors, alpha=0.4)


ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.title("Hierarchical DBSCAN w/ %d Clusters" % num_clusters)

for angle in range(0, 360):
    ax.view_init(30, angle)
    plt.draw()
    plt.pause(.001)


In [ ]:
#2D scatter plot of top 2 PC's
%matplotlib inline
plt.figure(figsize=(10, 10))

color_palette = sns.color_palette('bright', 8)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.8, 0.8, 0.8)
                  for x in clusterer.labels_]
cluster_member_colors = [sns.desaturate(x, p) for x, p in
                         zip(cluster_colors, clusterer.probabilities_)]
plt.scatter(allschoolsPCA.PC1, allschoolsPCA.PC2,  linewidth=0, c=cluster_member_colors, alpha=0.4)
plt.title("Hierarchical DBSCAN w/ %d Clusters" % (num_clusters + 1))

plt.show

In [ ]:
#dendogram of the clusters
%matplotlib inline
plt.figure(figsize=(10, 10))
clusterer.condensed_tree_.plot(select_clusters=True,
                               selection_palette=sns.color_palette('bright', 8))

In [ ]:
#create empty lists and dataframes for all the clusters
cluster1 = []
cluster1_df = pd.DataFrame(columns=('CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10'))
cluster2 = []
cluster2_df = pd.DataFrame(columns=('CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10'))
cluster3 = []
cluster3_df = pd.DataFrame(columns=('CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10'))
cluster4 = []
cluster4_df = pd.DataFrame(columns=('CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10'))
cluster5 = []
cluster5_df = pd.DataFrame(columns=('CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10'))
cluster6 = []
cluster6_df = pd.DataFrame(columns=('CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10'))
cluster7 = []
cluster7_df = pd.DataFrame(columns=('CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10'))
cluster8 = []
cluster8_df = pd.DataFrame(columns=('CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10'))

#pull out the index of each cluster (school name)
for i in sorted_ClusterList:
    if i[0] == 0:
        cluster1.append(i[1])
    elif i[0] == 1:
        cluster2.append(i[1])
    elif i[0] == 2:
        cluster3.append(i[1])
    elif i[0] == 3:
        cluster4.append(i[1])
    elif i[0] == 4:
        cluster5.append(i[1])
    elif i[0] == 5:
        cluster6.append(i[1])
    elif i[0] == 6:
        cluster7.append(i[1])
    elif i[0] == 7:
        cluster8.append(i[1])
        
#pull in the desired elements for each school in a cluster and put it in a dataframe
for school in cluster1:
    row = pd.DataFrame(all_schools.loc[[school], ['CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10']])
    cluster1_df = cluster1_df.append(row)

for school in cluster2:
    row = pd.DataFrame(all_schools.loc[[school], ['CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10']])
    cluster2_df = cluster2_df.append(row)
    
for school in cluster3:
    row = pd.DataFrame(all_schools.loc[[school], ['CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10']])
    cluster3_df = cluster3_df.append(row)
    
for school in cluster4:
    row = pd.DataFrame(all_schools.loc[[school], ['CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10']])
    cluster4_df = cluster4_df.append(row)
    
for school in cluster5:
    row = pd.DataFrame(all_schools.loc[[school], ['CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10']])
    cluster5_df = cluster5_df.append(row)
    
for school in cluster6:
    row = pd.DataFrame(all_schools.loc[[school], ['CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10']])
    cluster6_df = cluster6_df.append(row)
    
for school in cluster7:
    row = pd.DataFrame(all_schools.loc[[school], ['CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10']])
    cluster7_df = cluster7_df.append(row)
    
for school in cluster8:
    row = pd.DataFrame(all_schools.loc[[school], ['CONTROL', 'GRAD_DEBT_MDN_SUPP', 'MD_EARN_WNE_P10']])
    cluster8_df = cluster8_df.append(row)

In [ ]:
avg_md_debt = (cluster1_df.GRAD_DEBT_MDN_SUPP.mean(), cluster2_df.GRAD_DEBT_MDN_SUPP.mean(),
              cluster3_df.GRAD_DEBT_MDN_SUPP.mean(),cluster4_df.GRAD_DEBT_MDN_SUPP.mean(),
              cluster5_df.GRAD_DEBT_MDN_SUPP.mean(),cluster6_df.GRAD_DEBT_MDN_SUPP.mean(),
              cluster7_df.GRAD_DEBT_MDN_SUPP.mean(),cluster8_df.GRAD_DEBT_MDN_SUPP.mean())
avg_md_earn = (cluster1_df.MD_EARN_WNE_P10.mean(),cluster2_df.MD_EARN_WNE_P10.mean(),
              cluster3_df.MD_EARN_WNE_P10.mean(),cluster4_df.MD_EARN_WNE_P10.mean(),
              cluster5_df.MD_EARN_WNE_P10.mean(),cluster6_df.MD_EARN_WNE_P10.mean(),
              cluster7_df.MD_EARN_WNE_P10.mean(),cluster8_df.MD_EARN_WNE_P10.mean())

In [ ]:
plt.figure(figsize=(10,8))

ind = np.arange(len(avg_md_earn)) 
width = 0.35       
plt.bar(ind, avg_md_earn, width, label='Avg. Median Earnings', color='g')
plt.bar(ind + width, avg_md_debt, width, label='Avg. Median Debt', color='r')

plt.ylabel('Income/Debt')
plt.title('Clusters')

plt.xticks(ind + width / 2, ('Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6',
                            'Cluster 7', 'Cluster 8'))
plt.legend(loc='best')
plt.show()